✅ 현재 구조 요약
**사용자가 입력한 데이터 (user_data)**를 기반으로,

**3가지 질병 (당뇨병, 고혈압, 비만)**에 대한 예측 결과를 동시에 보여주고,

각 예측 결과에 대해:

예측 클래스 (정상, 주의, 고위험 등)

예측 확률

대표 feature 값 (HE_glu, HE_sbp2, BMI_calculated)

해당 값의 상태 분류 (정상/주의/고위험)

참고 범위

해석을 함께 출력합니다.

이후 **군집화(클러스터링)**를 통해 사용자 건강 특성을 분류하고자 합니다.

✅ 정리된 목표
목표	설명
1. 예측 결과 보고서 생성	각 질병에 대해 사용자 상태, 예측 결과, 주요 변수 해석 출력
2. 상태 분류 기준 정리	대표 feature에 대한 분류 기준(정상/주의/고위험)을 기반으로 상태 자동 판별
3. 군집화 준비	예측 결과를 바탕으로 군집화에 활용할 health profile 생성
4. 가중치 기반 군집화 (선택)	세 질병 중 어떤 질병 위험이 높은지에 따라 가중치 반영 가능

✅ 1. 상태 분류 기준 함수 (자동 분류용)
이 기준을 코드화하면 다음과 같이 사용할 수 있습니다:

def classify_feature_status(feature_name, user_value):
    if feature_name == "HE_sbp2":
        if user_value <= 120:
            return "정상", "90 ~ 120", "121 ~ 139"
        elif 121 <= user_value <= 139:
            return "주의", "90 ~ 120", "121 ~ 139"
        else:
            return "고위험", "90 ~ 120", "121 ~ 139"

    elif feature_name == "HE_glu":
        if user_value <= 99:
            return "정상", "70 ~ 99", "100 ~ 125"
        elif 100 <= user_value <= 125:
            return "주의", "70 ~ 99", "100 ~ 125"
        else:
            return "고위험", "70 ~ 99", "100 ~ 125"

    elif feature_name == "BMI_calculated":
        if user_value < 18.5:
            return "낮은 신체", "18.5 ~ 22.9", "23.0 ~ 24.9 이상"
        elif 18.5 <= user_value <= 22.9:
            return "정상", "18.5 ~ 22.9", "23.0 ~ 24.9 이상"
        elif 23.0 <= user_value <= 24.9:
            return "과체중", "18.5 ~ 22.9", "23.0 ~ 24.9 이상"
        else:
            return "비만", "18.5 ~ 22.9", "23.0 ~ 24.9 이상"
    else:
        return "Unknown", "-", "-"


✅ 2. 예측 결과를 종합해 리포트 생성
위 함수를 활용하여 generate_disease_report() 내부를 아래처럼 확장하면 됩니다:


def generate_disease_report(user_name, disease_name, result, top_feature_name, user_value):
    status, normal_range, caution_range = classify_feature_status(top_feature_name, user_value)

    print(f"=== {disease_name} 예측 결과 ===")
    print(f"1. {user_name}, 건강은 {disease_name} 위험군입니다.")
    print(f"2. 건강 점수: {float(result['probabilities'][result['predicted_class'] - 1]) * 100:.1f}점 (예측 확률: {float(result['probabilities'][result['predicted_class'] - 1]) * 100:.1f}%)")
    print(f"   예측 결과: {result['class_name']} ({result['predicted_class']}등급)")
    print(f"   설명: {result['explanation']}")
    print(f"3. {top_feature_name} 값: {user_value} → 상태: {status}")
    print(f"   정상 기준: {normal_range}, 주의 기준: {caution_range}")
    print(f"   이 수치가 증가하면 {disease_name} 위험이 증가합니다.\n")


✅ 3. 군집화 준비 (예측 결과 기반 feature vector 생성)
군집화를 하려면 예측 결과를 다음과 같이 정량화해야 합니다:

cluster_vector = {
    "당뇨병_등급": result_dm['predicted_class'],  # 1~3
    "고혈압_등급": result_hp['predicted_class'],  # 1~4
    "비만_등급": result_obesity['predicted_class']  # 1~6
}

# 또는 상태를 one-hot 인코딩할 수도 있음
그 후, 여러 사용자의 cluster_vector를 기반으로 KMeans, DBSCAN, HDBSCAN 등의 군집화를 진행할 수 있습니다.


✅ 4. (선택) 가중치 기반 클러스터링
당뇨병 위험을 더 중요하게 보고 싶다면, 예를 들어:

cluster_vector_weighted = {
    "당뇨병_등급": result_dm['predicted_class'] * 2.0,  # 더 중요한 위험요소
    "고혈압_등급": result_hp['predicted_class'] * 1.0,
    "비만_등급": result_obesity['predicted_class'] * 1.2
}
이렇게 비율 조절도 가능합니다.


✅ 전체 구조도 예시
css
복사
편집
[사용자 입력]
      ↓
[예측 모델 (3가지)]
      ↓
[대표 변수 기준 상태 분류]
      ↓
[사용자 리포트 생성]
      ↓
[예측 결과 기반 벡터 생성]
      ↓
[사용자 군집화]
🔚 마무리 제안
리포트 출력은 위 구조를 함수화해서 자동화

top_feature_name, user_value는 매핑으로 자동 추출

군집화는 predicted_class들을 기준으로 벡터 만들고 KMeans 등 적용